In [1]:
import pandas as pd 
import pandera as pa

In [2]:
valores_ausentes = ['**', '###!', '####', '****', '*****', 'NULL']
df = pd.read_csv(
                "ocorrencia_2010_2020.csv", 
                sep=";", 
                parse_dates=["ocorrencia_dia"], 
                dayfirst=True, 
                na_values = valores_ausentes
) 


In [3]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo": pa.Column(pa.Int, required=False), #por padrão as colunas são setadas como True, mas podemos definir se ela é obrigatória ou não. No caso essa coluna não existe na nossa base
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True), #tamanha máximo e minimo de caracteres
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }

)


In [4]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,3:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5735,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5736,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5737,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5738,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00,0


In [16]:
filtro1 = df.ocorrencia_uf == 'CE'
filtro2 = df.total_recomendacoes > 0
dfceara = df.loc[filtro1 & filtro2]
dfceara

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
531,43721,43721,ACIDENTE,CRATEÚS,CE,SNWS,2010-12-15,9:00:00,4
956,45052,45052,ACIDENTE,IGUATU,CE,SNIG,2011-08-29,10:20:00,5
2070,48652,48652,ACIDENTE,GUARAMIRANGA,CE,SION,2013-05-30,15:00:00,2
2303,50095,50095,INCIDENTE GRAVE,ITAPIPOCA,CE,NaN,2013-10-15,14:30:00,2
2511,51075,51075,INCIDENTE,FORTALEZA,CE,SBFZ,2014-02-07,21:55:00,1
2652,51011,51011,ACIDENTE,AQUIRAZ,CE,NaN,2014-05-03,15:30:00,2
3233,53340,53340,ACIDENTE,CAUCAIA,CE,NaN,2015-06-26,19:38:00,8
3252,53360,53360,ACIDENTE,CAUCAIA,CE,NaN,2015-07-10,14:50:00,2
4233,77353,77353,ACIDENTE,ORÓS,CE,NaN,2017-11-10,15:43:00,2
4515,77977,77977,ACIDENTE,FORTALEZA,CE,NaN,2018-06-23,20:00:00,1


In [19]:
dfceara.groupby(['ocorrencia_cidade', 'total_recomendacoes']).size().sort_values(ascending=False)

ocorrencia_cidade  total_recomendacoes
FORTALEZA          1                      2
AQUIRAZ            2                      1
CAUCAIA            2                      1
                   8                      1
CHORÓ              2                      1
CRATEÚS            4                      1
GUARAMIRANGA       2                      1
IGUATU             5                      1
ITAPIPOCA          2                      1
ORÓS               2                      1
dtype: int64

In [18]:
dfceara.groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_cidade
CAUCAIA         10
IGUATU           5
CRATEÚS          4
AQUIRAZ          2
CHORÓ            2
FORTALEZA        2
GUARAMIRANGA     2
ITAPIPOCA        2
ORÓS             2
Name: total_recomendacoes, dtype: int64